In [ ]:
import os

import papermill as pm

In [ ]:
# username = "Fro116"
# source = "MAL"
username = "10"
source = "Training"

In [ ]:
data_path = "../../data/recommendations"
if not os.path.exists(data_path):
    os.mkdir(data_path)

In [ ]:
papermill_path = f"../../data/recommendations/{username}/papermill/"

In [ ]:
def run_ingress_notebook(name):
    outdir = os.path.join(papermill_path, "rec_egress")
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    pm.execute_notebook(
        f"../RecIngress/{name}.ipynb",
        f"{outdir}/{name}.ipynb",
        parameters=dict(username=username, source=source),
    )

## RecIngress

In [ ]:
run_ingress_notebook("InitializeRecommendations")

In [ ]:
run_ingress_notebook("FetchUserAnimeList")

In [ ]:
run_ingress_notebook("ImportUserAnimeList")

In [ ]:
run_ingress_notebook("ProcessUserAnimeList")

In [ ]:
run_ingress_notebook("GenerateSplits")

In [ ]:
run_ingress_notebook("CompressUserAnimeList")

## RecEgress

In [ ]:
def run_inference_notebook(name):
    cwd = os.getcwd()
    fields = name.split("/")
    outdir = os.path.abspath(
        os.path.join(papermill_path, "inference_alphas", "/".join(fields[:-1]).lower())
    )
    if not os.path.exists(outdir):
        os.makedirs(outdir, exist_ok=True)
    try:
        os.chdir("../InferenceAlphas")
        for subdir in fields[:-1]:
            os.chdir(subdir)
        pm.execute_notebook(
            f"{fields[-1]}.ipynb",
            os.path.join(outdir, f"{fields[-1]}.ipynb"),
            parameters=dict(username=username, outdir=outdir),
        )
    finally:
        os.chdir(cwd)

In [ ]:
run_inference_notebook("ResetAlphas")

In [ ]:
run_inference_notebook("Nondirectional/RunAllScripts")

In [ ]:
run_inference_notebook("Explicit/RunAllScripts")

In [ ]:
run_inference_notebook("Neural/RunAllScripts")

In [ ]:
# cwd = os.getcwd()
# try:
#     os.chdir("../InferenceAlphas")
#     pm.execute_notebook(
#         "RunAllAlphas.ipynb",
#         f"{data_path}/RunAllAlphas.ipynb",
#         parameters=dict(username=username),
#     )
#     os.chdir("../DisplayRecs")
#     if source == "Training":
#         pm.execute_notebook(
#             "TrainingServingSkew.ipynb",
#             f"{data_path}/TrainingServingSkew.ipynb",
#             parameters=dict(username=username),
#         )
#     else:
#         pm.execute_notebook(
#             "Recommendations.ipynb",
#             f"{data_path}/{username}/Recommendations.ipynb",
#             parameters=dict(username=username),
#         )
# finally:
#     os.chdir(cwd)

In [ ]:
print("Success!")